In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

# Visualize US Data

In [ ]:
from geonotebook.wrappers import RasterData

In [ ]:
rd = RasterData("geotrellis://http://localhost:8899/tiles/weld11")

In [ ]:
M.add_layer(rd)

# Subset with a rectangle

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyproj import Proj, transform
from shapely.geometry import Polygon
from geopyspark.geotrellis.catalog import S3Catalog
from geopyspark.geopycontext import GeoPyContext
from geopyspark.avroregistry import AvroRegistry


In [ ]:
def latLongToWebMercator(lat_long_coords):
    in_proj  = Proj("+init=EPSG:4326")
    out_proj = Proj("+init=EPSG:3857")
    return [transform(in_proj, out_proj, x, y) for (x, y) in lat_long_coords]

def webMercatorToLatLong(web_merc_coords):
    out_proj  = Proj("+init=EPSG:4326")
    in_proj = Proj("+init=EPSG:3857")
    return [transform(in_proj, out_proj, x, y) for (x, y) in web_merc_coords]    


In [ ]:
# If sparkcontext already exists use it
try:
    geopysc = GeoPyContext(master='local[*]', appName='data_frame_test')
except ValueError:
    pass

catalog = S3Catalog(geopysc)


In [ ]:
polygon = Polygon(
    latLongToWebMercator(list(M.layers.annotation.rectangles[0].exterior.coords)))
print(latLongToWebMercator(list(M.layers.annotation.rectangles[0].exterior.coords)))

In [ ]:
key_type = 'spatial'
value_type = 'singleband'
bucket = "kitware-catalog"
prefix = 'full-catalog'
layer_name = 'weld11'
layer_zoom = 13

(rdd, schema, metadata) = catalog.query(key_type, value_type, bucket, 
                                prefix, layer_name, layer_zoom, polygon)
print(metadata)

In [ ]:
ta = rdd.collect()

In [ ]:
# Get extent of rdd in lat long
extent = metadata['extent']
rdd_extent = webMercatorToLatLong([(extent['xmin'],
                                   extent['ymin']), 
                                   (extent['xmax'],
                                   extent['ymax'])])


# Investigate

In [ ]:
def stitch_tilearrays(ta):
    arr = None
    c_arr = None
    prev_col = None

    for key, _ta in sorted(ta, key=lambda x: (x[0].col, x[0].row)):
        # Build up the column
        if prev_col == None:
            c_arr = _ta
        elif prev_col == key.col:
            c_arr = np.vstack([c_arr, _ta])
        # We've finished with the column,  add it
        # to the array
        else:
            if arr is None:
                arr = c_arr
            else:
                arr = np.hstack([arr, c_arr])

            c_arr = _ta
        prev_col = key.col

    return np.hstack([arr, c_arr])


In [ ]:
plt.imshow(stitch_tilearrays(ta))